<a href="https://colab.research.google.com/github/jsn-li/evadb/blob/main/Home_Price_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Home Price Forecasting with EvaDB

## Setup
We must install Postgres and EvaDB and integrate them. Then, we load our database with our home sale data.

### Install and Initialize Postgres

In [1]:
!apt install postgresql
!service postgresql start
!sudo -u postgres psql -c "CREATE USER admin WITH SUPERUSER PASSWORD 'password'"
!sudo -u postgres psql -c "CREATE DATABASE evadb"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libcommon-sense-perl libjson-perl libjson-xs-perl libtypes-serialiser-perl logrotate netbase
  postgresql-14 postgresql-client-14 postgresql-client-common postgresql-common ssl-cert sysstat
Suggested packages:
  bsd-mailx | mailx postgresql-doc postgresql-doc-14 isag
The following NEW packages will be installed:
  libcommon-sense-perl libjson-perl libjson-xs-perl libtypes-serialiser-perl logrotate netbase
  postgresql postgresql-14 postgresql-client-14 postgresql-client-common postgresql-common ssl-cert
  sysstat
0 upgraded, 13 newly installed, 0 to remove and 18 not upgraded.
Need to get 18.3 MB of archives.
After this operation, 51.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 logrotate amd64 3.19.0-1ubuntu1.1 [54.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd6

### Install and Configure EvaDB

In [ ]:
!pip install "evadb[postgres,forecasting] @ git+https://github.com/georgia-tech-db/evadb.git@68265d3b138babfe4a20091bc5fa7a67b56072f5"

  Cloning https://github.com/georgia-tech-db/evadb.git (to revision 68265d3b138babfe4a20091bc5fa7a67b56072f5) to /tmp/pip-install-e9a8geut/evadb_d9c463996a2d42df8aeb35ef0dfba77b
  Running command git clone --filter=blob:none --quiet https://github.com/georgia-tech-db/evadb.git /tmp/pip-install-e9a8geut/evadb_d9c463996a2d42df8aeb35ef0dfba77b
  Running command git rev-parse -q --verify 'sha^68265d3b138babfe4a20091bc5fa7a67b56072f5'
  Running command git fetch -q https://github.com/georgia-tech-db/evadb.git 68265d3b138babfe4a20091bc5fa7a67b56072f5


In [ ]:
import evadb
cursor = evadb.connect().cursor()

params = {
    "user": "admin",
    "password": "password",
    "host": "localhost",
    "port": "5432",
    "database": "evadb",
}
query = f"CREATE DATABASE postgres_data WITH ENGINE = 'postgres', PARAMETERS = {params};"
cursor.query(query).df()

### Load the Home Sales Dataset


In [ ]:
## drop table if exists
cursor.query("""
  USE postgres_data {
    DROP TABLE IF EXISTS home_sales
  }
""").df()

## create home sale data table
cursor.query("""
  USE postgres_data {
    CREATE TABLE home_sales(date_recorded VARCHAR(64), town VARCHAR(64), sale_amount DECIMAL, property_type VARCHAR(64), residential_type VARCHAR(64))
  }
""").df()

## populate home sale table with dataset
!wget -O /connecticut.csv https://raw.githubusercontent.com/jsn-li/evadb/main/connecticut.csv
cursor.query("""
  USE postgres_data {
    COPY home_sales(date_recorded, town, sale_amount, property_type, residential_type)
    FROM '/connecticut.csv'
    DELIMITER ',' CSV HEADER
  }
""").df()

## Train the Model using EvaDB


In [ ]:
## Set parameters!
TOWN="Stamford"  # town to forecast pricing for. valid values: any town in Connecticut
PROPERTY_TYPE="Vacant Land"  # what type of property to forecast prices for. valid values: "Condo", "Single Family", "Two Family", "Three Family", "Vacant Land", "Commerical", "Apartments"
FREQUENCY="M"  # prediction granularity. valid values: D, W, M, Y (day, week, month, year)
HORIZON=3  # how many predictions to make (e.g. 3 for 3 months (when prediction frequency is set to M))

## Train model
cursor.query(f"""
  CREATE OR REPLACE FUNCTION Forecast FROM
    (
      SELECT town, property_type, residential_type, date_recorded, sale_amount
      FROM postgres_data.home_sales
      WHERE town = '{TOWN}' AND (
        (property_type = '{PROPERTY_TYPE}')
        OR
        (property_type = 'Residential' AND residential_type = '{PROPERTY_TYPE}')
      )
    )
  TYPE Forecasting
  PREDICT 'sale_amount'
  TIME 'date_recorded'
  FREQUENCY '{FREQUENCY}'
  HORIZON {HORIZON}
""").df()

/usr/local/lib/python3.10/dist-packages/evadb/third_party/databases/postgres/postgres_handler.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tables_df = pd.read_sql_query(query, self.connection)
/usr/local/lib/python3.10/dist-packages/evadb/third_party/databases/postgres/postgres_handler.py:111: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  columns_df = pd.read_sql_query(query, self.connection)
/usr/local/lib/python3.10/dist-packages/evadb/third_party/databases/postgres/postgres_handler.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please conside

Training, please wait...


,0
0,Function Forecast overwritten.


## Retrieve Forecast Result

In [ ]:
cursor.query("SELECT Forecast();").df()

,forecast.date_recorded,forecast.sale_amount
0,2021-09-30,1423865.625
1,2021-10-31,1400000.000
2,2021-11-30,850000.000


import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(_df_0, *['forecast.sale_amount'], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_1, *['forecast.date_recorded', 'forecast.sale_amount', None], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_2, *['forecast.date_recorded', 'count()', None], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_3, *['forecast.sale_amount'], **{})
chart